In [1]:
pip install keras-tuner

     |████████████████████████████████| 71kB 6.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=1f8c0ae311b95249197458612ca6ba27aaa9651c9401f02ccf96b0c7d09019ce
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15357 sha256=7fbed59efb6ad54b20c4ced6dd7f42ae86418ef16b6014792dabf541ade732a0
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
from tensorflow import keras
from kerastuner.tuners import BayesianOptimization
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATA_PATH = '/content/drive/MyDrive/data.json'

In [5]:
def load_data(data_path):
    
    with open(data_path, "r") as fp:
        data = json.load(fp)
    
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X,y 

In [6]:
def prepare_datasets(test_size, validation_size):
    
    # load data
    X, y = load_data(DATA_PATH)
    
    # create trai/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    # create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [7]:
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

In [8]:
input_shape = (X_train.shape[1], X_train.shape[2]) # 130m 13
#model = build_model(input_shape)
input_shape

(130, 13)

In [9]:
def build_model(hp):
    """Generates RNN-LSTM model"""
    
    # create model
    model = keras.Sequential()
    
    # 2 LSTM layers
    model.add(keras.layers.LSTM(units=hp.Int('units',min_value=32,max_value=512,step=32),
                                input_shape=input_shape, 
                                return_sequences=True))
    model.add(keras.layers.LSTM(units=hp.Int('units',min_value=32,max_value=512,step=32)))
    
    # dense layer
    model.add(keras.layers.Dense(units=hp.Int('units',min_value=32,max_value=512), activation='relu'))
    model.add(keras.layers.Dropout(rate=hp.Float('dropout', 0, 0.5, step=0.1, default=0.3)))
    
    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))
              
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])), 
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    
    return model

In [10]:
import time
LOG_DIR = f"{int(time.time())}"

In [13]:
bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=1,
    executions_per_trial=1,
    directory = LOG_DIR,
    project_name='kerastuner_bayesian_poc',
    overwrite=True)

bayesian_opt_tuner.search(X_train, y_train,epochs=50,
                          batch_size=1,
     validation_data=(X_validation, y_validation),
     #validation_split=0.2,
                    verbose=1)


bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)
model = bayes_opt_model_best_model[0]

Trial 1 Complete [01h 23m 20s]
val_accuracy: 0.8638184070587158

Best val_accuracy So Far: 0.8638184070587158
Total elapsed time: 01h 23m 20s
INFO:tensorflow:Oracle triggered exit


In [14]:
model.save('tuned_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: tuned_model/assets


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 130, 224)          213248    
_________________________________________________________________
lstm_1 (LSTM)                (None, 224)               402304    
_________________________________________________________________
dense (Dense)                (None, 224)               50400     
_________________________________________________________________
dropout (Dropout)            (None, 224)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2250      
Total params: 668,202
Trainable params: 668,202
Non-trainable params: 0
_________________________________________________________________


In [18]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

79/79 - 1s - loss: 0.7948 - accuracy: 0.8490

Test accuracy: 0.8490188121795654
